<a href="https://colab.research.google.com/github/edgardominguez23/Metrics/blob/main/TI_1_Practica3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 3

**Nombre:** Edgar Alan Dominguez Murillo  
**e-mail:** edgar.dominguez@alumnos.udg.mx

## MODULES

In [3]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance
from scipy.stats import norm

## CLASSES

In [7]:
################## http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## FUNCTIONS

In [8]:
###############################################################################################
# Turning angle
# This function calculates the turning angle between three consecutive positions
###############################################################################################
def turning_angle(pos_a, pos_b, pos_c):
    """
    Arguments:
        pos_a: First position coordinates
        pos_b: Second position coordinates
        pos_c: Third position coordinates
    Returns:
        theta: Turning angle
    """
    vec_ab = np.array(pos_b) - np.array(pos_a)
    norm_ab = np.linalg.norm(vec_ab)

    vec_bc = np.array(pos_c) - np.array(pos_b)
    norm_bc = np.linalg.norm(vec_bc)

    dot_p = np.dot(vec_ab, vec_bc)

    # Nota: Evitar division por cero con np.finfo(float).eps
    cos_theta = dot_p / (norm_ab * norm_bc + np.finfo(float).eps)

    # Angle orientation
    cross_p = np.cross(vec_ab, vec_bc)
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1

    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta



## Actividad 1: Path Length - (BM1 vs BM2 vs CRW) (4 pts)

* Implementar función que genere **Brownian Motions** (BM) utilizando **pandas**.
* Implementar función que genere **Correlated Random Walks** (CRW) utilizando pandas.
* Implementar una función alternativa a las ya disponibles en los distintos modulos de python que calcule los valores de la curva de **path length** de una trayectoria.
* Guardar los valores de la métrica en un Data Frame de **pandas**.
* Visualizar con **plotly**.

In [9]:
# Load existing trajectories to test your implementation
# BM speed = 3
BM_2d_df_3 = pd.read_csv('trajectories/brownian_3.csv')

# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories to test your implementation
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [10]:
# Define your function to compute path length for given trajectory

## start - Add your code here
def consecutive_distance(trajectory):
  dis = np.array([distance.euclidean(trajectory.iloc[i-1], trajectory.iloc[i]) for i in range(1, trajectory.shape[0])])
  # Calculo del path length
  path_length_df = np.cumsum(dis)

  return path_length_df
## end - Add your code here

In [11]:
# Get Path length calling the function
PL_BM_3 = consecutive_distance(BM_2d_df_3)
PL_BM_3 = pd.concat([pd.DataFrame({'x_pos': np.arange(len(PL_BM_3))+1}), pd.DataFrame({'y_pos': PL_BM_3})], axis=1)

# Get Path length calling the function
PL_BM_6 = consecutive_distance(BM_2d_df_6)
PL_BM_6 = pd.concat([pd.DataFrame({'x_pos': np.arange(len(PL_BM_6))+1}), pd.DataFrame({'y_pos': PL_BM_6})], axis=1)

# Get Path length calling the function
PL_CRW_6 = consecutive_distance(CRW_2d_df_9)
PL_CRW_6 = pd.concat([pd.DataFrame({'x_pos': np.arange(len(PL_CRW_6))+1}), pd.DataFrame({'y_pos': PL_CRW_6})], axis=1)

In [12]:
# Plotting
# Init figure
fig_path_length = go.Figure()

# First trace BM1
## start - Add your code here
fig_path_length.add_trace(go.Scatter(
    x = PL_BM_3.x_pos,
    y = PL_BM_3.y_pos,
    name = 'path length BM 3',
    showlegend = True
))
## end - Add your code here

# Second trace BM2
## start - Add your code here
fig_path_length.add_trace(go.Scatter(
    x = PL_BM_6.x_pos,
    y = PL_BM_6.y_pos,
    line = dict(width = 6),
    name = 'path length BM 6',
    showlegend = True
))
## end - Add your code here

# Third trace CRW
## start - Add your code here
fig_path_length.add_trace(go.Scatter(
    x = PL_CRW_6.x_pos,
    y = PL_CRW_6.y_pos,
    name = 'path length CRW 6',
    showlegend = True
))
## end - Add your code here

fig_path_length.show()

## Actividad 2: Mean Squared Displacement - (Brownian vs CRW) (4 pts)

* Generar una trayectoria tipo **BM** y una **CRW**.
* Implementar una función que calcule los valores de la curva de **mean squared displacement** de una trayectoria.
* Guardar metricas en Pandas Data Frame.
* Visualizar con **plotly**.

In [13]:
# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6, c = 0.9
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [14]:
# Show trajectories
# Init figure
fig_3d = go.Figure()

# Plot trajectory in 3-D space
fig_3d.add_trace(
    go.Scatter3d(x = BM_2d_df_6.x_pos,
                 y = BM_2d_df_6.y_pos,
                 z = BM_2d_df_6.index,
                 marker = dict(size=2),
                 line = dict(color='blue', width=2),
                 mode = 'lines',
                 name = 'BM 2d',
                 showlegend = True))


fig_3d.add_trace(
    go.Scatter3d(x = CRW_2d_df_9.x_pos,
                 y = CRW_2d_df_9.y_pos,
                 z = CRW_2d_df_9.index,
                 marker = dict(size=2),
                 line = dict(color='red', width=2),
                 mode = 'lines',
                 name = 'CRW 2d',
                 showlegend = True))

fig_3d.show()

In [15]:
# Define your function to compute Mean Squared Displacement for given trajectory

## start - Add your code here
def msd_curve(trajectory):
  dx = np.diff(np.square(trajectory['x_pos']))
  dy = np.diff(np.square(trajectory['y_pos']))

  squared_displacement = dx + dy

  MSD_df = np.cumsum(squared_displacement)
  return MSD_df
## end - Add your code here

In [16]:
# Get Mean Squared Displacement calling the function
MSD_BM = msd_curve(BM_2d_df_6)
MSD_BM = pd.concat([pd.DataFrame({'x_pos': np.arange(0, len(MSD_BM))}), pd.DataFrame({'y_pos': MSD_BM})], axis=1)

# Get Mean Squared Displacement calling the function
MSD_CRW = msd_curve(CRW_2d_df_9)
MSD_CRW = pd.concat([pd.DataFrame({'x_pos': np.arange(0, len(MSD_CRW))}), pd.DataFrame({'y_pos': MSD_CRW})], axis=1)

In [17]:
# Init figure
fig_path_length = go.Figure()

# first trace MSD_BM
## start - Add your code here
fig_path_length.add_trace(go.Scatter(
    x = MSD_BM.x_pos,
    y = MSD_BM.y_pos,
    name = 'MSB BM',
    showlegend = True
))
## end - Add your code here


# Second trace MSD_CRW
## start - Add your code here
fig_path_length.add_trace(go.Scatter(
    x = MSD_CRW.x_pos,
    y = MSD_CRW.y_pos,
    name = 'MSB CRW',
    showlegend = True
))
## end - Add your code here


fig_path_length.show()

## Actividad 3: Turning-angle Distribution - (Dist. origen vs Dist. observada) (6 pts)

* Generar **CRWs** con dos exponentes diferentes
* Guardar trayectorias en **Pandas** Data Frame
* Implementar una función que calcule los valores de **turning angle** de una trayectoria.
* Comparar en gráfica distribución origen vs distribución observada (Histograma)
* Visualizar con **plotly**

In [18]:
# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.6]
CRW_2d_df_6 = pd.read_csv('trajectories/crw_6_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.9]
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [19]:
# Define your function to compute Turning Angles for given trajectory

## start - Add your code here
def calculate_turning_angles(trajectory):
  ta_df = []

  for i in range(len(trajectory) - 2):
      pos_a = trajectory.iloc[i]
      pos_b = trajectory.iloc[i + 1]
      pos_c = trajectory.iloc[i + 2]

      # Calculate turning angle between pos_a, pos_b, pos_c
      angle = turning_angle(pos_a, pos_b, pos_c)

      # Append the turning angle to the list
      ta_df.append(angle)

  return ta_df
## end - Add your code here

In [20]:
# Get Turning Angles calling the function
ta_CRW_2d_df_6 = calculate_turning_angles(CRW_2d_df_6)

# Get Turning Angles calling the function
ta_CRW_2d_df_9 = calculate_turning_angles(CRW_2d_df_9)

In [21]:
# Check documentation
# https://plotly.com/python/histograms/

# PLot histogram
fig = go.Figure()

# Histogram turning angle CRW_2d_df_6
## start - Add your code here
fig.add_trace(go.Histogram(
    x=ta_CRW_2d_df_6,
    name='observed 0.6',
    histnorm='probability density'
))
## end - Add your code here

# Histogram turning angle CRW_2d_df_9
## start - Add your code here
fig.add_trace(go.Histogram(
    x=ta_CRW_2d_df_9,
    name='observed 0.9',
    histnorm='probability density'
))
## end - Add your code here

# Add origin distributions
## start - Add your code here
x_values = np.linspace(min(ta_CRW_2d_df_6), max(ta_CRW_2d_df_6), 1000)
y_values = np.array([wrapcauchy.pdf(abs(i), 0.6) for i in x_values])

fig.add_trace(go.Scatter(
    x=x_values,
    y=y_values,
    mode='lines',
    name='cauchy 0.6',
    line=dict(color='blue', width=2)
))

x_values = np.linspace(min(ta_CRW_2d_df_9), max(ta_CRW_2d_df_9), 1000)
y_values = np.array([wrapcauchy.pdf(abs(i), 0.9) for i in x_values])

fig.add_trace(go.Scatter(
    x=x_values,
    y=y_values,
    mode='lines',
    name='cauchy 0.9',
    line=dict(color='red', width=2)
))
## end - Add your code here

fig.show()

## Actividad 4: Step-length Distribution - (Dist. origen vs Dist. observada) (6 pts)

* Implementar función que genere **Lévy Walks** (LW) utilizando pandas.
* Guardar trayectorias en Pandas Data Frame.
* Implementar una función que calcule los valores de **step lenght** de una trayectoria.
* Guardar trayectorias en **pandas** Data Frame.
* Obtener **Step-length** distribution.
* Comparar en gráfica distribución origen vs distribución observada (Histograma).
* Visualizar con plotly.

In [270]:
# Load existing trajectories to test your implementation
# Levy speed = 6
#levy_stable [alpha=1.0, beta=1.0, loc=3.0]
Levy_2d_df_1 = pd.read_csv('trajectories/levy_6_1.csv')

# Load existing trajectories to test your implementation
# Levy speed = 6
# levy_stable [alpha=0.7, beta=1.0, loc=3.0]
Levy_2d_df_7 = pd.read_csv('trajectories/levy_6_7.csv')

In [271]:
# Define your function to compute Step lengths for given trajectory

## start - Add your code here
def compute_step_lengths(trajectory):
    sl_df = []
    for i in range(1, len(trajectory)):
        x1, y1 = trajectory.iloc[i - 1]
        x2, y2 = trajectory.iloc[i]
        distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

        sl_df.append(distance)
    return sl_df
## end - Add your code here

In [297]:
# Get Step lengths calling the function
sl_Levy_2d_df_1 = compute_step_lengths(Levy_2d_df_1.head(1000))

# Get Step lengths calling the function
sl_Levy_2d_df_7 = compute_step_lengths(Levy_2d_df_7.head(1000))

In [299]:
# PLot histogram
fig = go.Figure()

# Histogram step-length Levy_2d_df_1
## start - Add your code here
r = levy_stable.rvs(1.0, 1.0, loc=3.0, size=1000)
r = np.clip(r, 0, 50)

fig.add_trace(go.Histogram(
    x=r,
    name='observed alpha 1.0 beta 1.0',
    histnorm='probability density'
))
## end - Add your code here


# Histogram step-length Levy_2d_df_7
## start - Add your code here
r = levy_stable.rvs(0.7, 1.0, loc=3.0, size=1000)
r = np.clip(r, 0, 50)

fig.add_trace(go.Histogram(
    x=r,
    name='observed alpha 0.7 beta 1.0',
    histnorm='probability density'
))
## end - Add your code here

# Add origin distributions
## start - Add your code here

#sl_1 = np.clip(sl_Levy_2d_df_1, 0, 50)

x_values = np.linspace(0, 50, 1000)
y_values = np.array([levy_stable.pdf(i, alpha=1.0, beta=1.0, loc=3.0) for i in x_values])

fig.add_trace(go.Scatter(
    x=x_values,
    y=y_values,
    mode='lines',
    name='Levy alpha 1.0',
    line=dict(color='blue', width=2)
))
## end - Add your code here

# Add origin distributions
## start - Add your code here
#sl_7 = np.clip(sl_Levy_2d_df_7, 0, 50)

#x_values = np.linspace(0, max(sl_7), 1000)
x_values = np.linspace(0, 50, 1000)
y_values = np.array([levy_stable.pdf(i, alpha=0.7, beta=1.0, loc=3.0) for i in x_values])

fig.add_trace(go.Scatter(
    x=x_values,
    y=y_values,
    mode='lines',
    name='Levy alpha 0.7',
    line=dict(color='red', width=2)
))
## end - Add your code here


fig.show()